In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import StratifiedKFold
import eli5

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# #Merged User info, activity log and training and testing data on user-merchant pairs
# df = pd.read_csv("/Users/Vishal/Project_1/final.csv")

# #Replace null values in columns with corresponding values

# df['age_range'] = df['age_range'].fillna(0)
# df['label'] = df['label'].fillna(2)
# df['gender'] = df['gender'].fillna(2)
# df['brand_id'] = df['brand_id'].fillna(0)

# # Convert time stamp to Year-Month-Date format
# df['time_stamp'] = pd.to_datetime(df['time_stamp'], format='%m%d', errors='coerce')

In [4]:
#Training data from format 1
df_train = pd.read_csv("/Users/Vishal/Project_1/data_format1/train_format1.csv")

# LOAD FEATURES

In [55]:
df_combined = pd.read_csv("/Users/Vishal/Project_1/df_combined.csv")

In [56]:
#Sort
df_combined = df_combined.sort_values(['user_id','seller_id'])
df_train = df_train.sort_values(['user_id','seller_id'])

final_feat = (df_combined.drop(['user_id', 'seller_id', 'label','age_gender', 'user_merchant_pair'], axis=1))

final_label = df_train['label']

# MODEL

# 5-FOLD STRATIFIED SPLIT 

In [57]:
def model_xgboost(x, y):
    
    model = xgb.XGBClassifier(max_depth = 6)
    
    skf = StratifiedKFold(n_splits = 5, random_state = 2)
    
    AUC_valid = []
    AUC_train = []
    
    for train_index, valid_index in skf.split(x, y):
    
        #print("TRAIN:", train_index, "VALID:", valid_index)
    
        X_train, X_valid = x.iloc[train_index], x.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    
        model.fit(X_train, y_train, eval_metric='auc')
        
        auc_valid = roc_auc_score(y_valid, model.predict_proba(X_valid)[:,1])
        #print(auc_valid)
        AUC_valid.append(auc_valid)
    
        auc_train = roc_auc_score(y_train, model.predict_proba(X_train)[:,1])
        #print(auc_train)
        AUC_train.append(auc_train)
        
        print('Iteration Complete!!')
        
    def mean(z):
        return sum(z)/len(z)
    
    print("Mean auc for validation = ", mean(AUC_valid), "\t Mean auc for valid baseline = 0.6498")
    print("Mean auc for train = ", mean(AUC_train), "\t Mean auc for train baseline = 0.6629")
        
    feat_imp_abs = model.feature_importances_
    feat_imp_abs = pd.Series(data=feat_imp_abs,dtype='float')
    feat_imp_norm = feat_imp_abs.div(feat_imp_abs.max())*100    
        
    feature_importance = list(final_feat)
    feature_importance = pd.DataFrame(feature_importance)
    feature_importance['importance_abs'] = feat_imp_abs
    feature_importance['importance_norm'] = feat_imp_norm
    feature_importance = feature_importance.rename(columns={0: "feature"})
    
    print(feature_importance.sort_values('importance_norm'))

In [58]:
# xgb_model = xgb.XGBClassifier(max_depth=3)

In [59]:
# # Max depth 3
# xgb_model.fit(X_train, y_train)

In [60]:
# roc_auc_score(y_test, xgb_model.predict_proba(X_test)[:,1])

In [61]:
# # Max depth 4
# xgb_model = xgb.XGBClassifier(max_depth=4)

# xgb_model.fit(X_train, y_train)


In [62]:
# roc_auc_score(y_test, xgb_model.predict_proba(X_test)[:,1])

In [63]:
# # Max depth 5
# xgb_model = xgb.XGBClassifier(max_depth=5)

# xgb_model.fit(X_train, y_train)


In [64]:
# roc_auc_score(y_test, xgb_model.predict_proba(X_test)[:,1])

In [65]:
# # Max depth 6
# xgb_model = xgb.XGBClassifier(max_depth=6)

# xgb_model.fit(X_train, y_train)


In [66]:
# roc_auc_score(y_test, xgb_model.predict_proba(X_test)[:,1])

# MODEL RESULTS

In [67]:
model_xgboost(final_feat,final_label)

Iteration Complete!!
Iteration Complete!!
Iteration Complete!!
Iteration Complete!!
Iteration Complete!!
Mean auc for validation =  0.6784846625584459 	 Mean auc for valid baseline = 0.6498
Mean auc for train =  0.7938491879476313 	 Mean auc for train baseline = 0.6629
                                            feature  importance_abs  importance_norm
137                               userselleronsale1        0.000000         0.000000
138                               userselleronsale2        0.000000         0.000000
134                           usersellerbeforesale1        0.000000         0.000000
79                         unique_users_gender_norm        0.000000         0.000000
151                         seller_count_of_label-1        0.000000         0.000000
78                              gender_avg_purchase        0.000000         0.000000
164         ratio_item_int_union_user_before_seller        0.000000         0.000000
152          brand_intersection_user_before_on_sal

In [ ]:
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report
# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import roc_curve
# import seaborn as sns
# import matplotlib.pyplot as plt

# def roc_auc(X_test,y_test,y_pred,model):
# #    confusion_mat = confusion_matrix(y_test, y_pred)
# #    print('Confusion_Matrix: ',confusion_mat,sep='\n')
# #    print()
# #    print('Accuracy of classifier on test set: {:.2f}'.format(model.score(X_test, y_test)))
# #    print()
   
# #    # Classification Report
# #    print('Classification Report: ')
# #    print(classification_report(y_test, y_pred))
# #    y_score = model.predict_proba(X_test)[:,1]              # Predicted probability score
# #    fpr, tpr, thresholds = roc_curve(y_test, y_score)
# #    plt.figure()
   
#    # AuC Score & plotting of AuC curve
#    plt.plot(fpr, tpr, label='AuC score (area = %0.2f)' % roc_auc_score(y_test, y_score))
#    plt.plot([0, 1], [0, 1],'r--')
#    plt.xlim([0.0, 1.0])
#    plt.ylim([0.0, 1.05])
#    plt.xlabel('False Positive Rate')
#    plt.ylabel('True Positive Rate')
#    plt.title('Receiver operating characteristic')
#    plt.legend(loc="lower right")

In [ ]:
# roc_auc(xval1,yval1,y_pred,regressor)

# LightGBM

In [18]:
import lightgbm as lgb

/Users/Vishal/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [19]:
X_train, X_test, y_train, y_test = train_test_split(final_feat, final_label, stratify=final_label, test_size=.2, random_state=42)


In [20]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [21]:
params = {'max_depth' : 10,
    'num_leaves': 5,
    'metric': ['l1', 'l2'],
    'verbose': -1
}

In [22]:
# Training

In [23]:
evals_result = {}  # to record eval results for plotting
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=[lgb_train, lgb_test],
                evals_result=evals_result,
                verbose_eval=10)

[10]	training's l1: 0.113915	training's l2: 0.0567144	valid_1's l1: 0.113904	valid_1's l2: 0.0567737
[20]	training's l1: 0.113414	training's l2: 0.0564414	valid_1's l1: 0.113399	valid_1's l2: 0.0565236
[30]	training's l1: 0.113078	training's l2: 0.056262	valid_1's l1: 0.11309	valid_1's l2: 0.0563785
[40]	training's l1: 0.11284	training's l2: 0.0561205	valid_1's l1: 0.112849	valid_1's l2: 0.0562719
[50]	training's l1: 0.112619	training's l2: 0.0559909	valid_1's l1: 0.112623	valid_1's l2: 0.0561551
[60]	training's l1: 0.112427	training's l2: 0.0558787	valid_1's l1: 0.112424	valid_1's l2: 0.0560675
[70]	training's l1: 0.112265	training's l2: 0.0557857	valid_1's l1: 0.112274	valid_1's l2: 0.0559988
[80]	training's l1: 0.112124	training's l2: 0.0557013	valid_1's l1: 0.112161	valid_1's l2: 0.0559422
[90]	training's l1: 0.112011	training's l2: 0.0556372	valid_1's l1: 0.112073	valid_1's l2: 0.0559084
[100]	training's l1: 0.111906	training's l2: 0.0555748	valid_1's l1: 0.111998	valid_1's l2: 0.

In [24]:
y_pred = gbm.predict(X_test)

In [25]:
print(roc_auc_score(y_test, gbm.predict(X_test)))

0.6623238745039624


In [26]:
def render_metric(metric_name):
    ax = lgb.plot_metric(evals_result, metric=metric_name, figsize=(10, 5))
    plt.show()

In [27]:
import matplotlib.pyplot as plt

%matplotlib inline

try:
    # To enable interactive mode you should install ipywidgets
    # https://github.com/jupyter-widgets/ipywidgets
    from ipywidgets import interact, SelectMultiple
    INTERACTIVE = True
except ImportError:
    INTERACTIVE = False

In [28]:
if INTERACTIVE:
    # create widget to switch between metrics
    interact(render_metric, metric_name=params['metric'])
else:
    render_metric(params['metric'][0])

interactive(children=(Dropdown(description='metric_name', options=('l1', 'l2'), value='l1'), Output()), _dom_c…

In [29]:
def render_plot_importance(importance_type, max_features=10,
                           ignore_zero=True, precision=3):
    ax = lgb.plot_importance(gbm, importance_type=importance_type,
                             max_num_features=max_features,
                             ignore_zero=ignore_zero, figsize=(12, 8),
                             precision=precision)
    plt.show()

In [30]:

if INTERACTIVE:
    # create widget for interactive feature importance plot
    interact(render_plot_importance,
             importance_type=['split', 'gain'],
             max_features=(1, X_train.shape[-1]),
             precision=(0, 10))
else:
    render_plot_importance(importance_type='split')

interactive(children=(Dropdown(description='importance_type', options=('split', 'gain'), value='split'), IntSl…

# Model without seller features

In [31]:
# Without Seller monthly features

df_combined.iloc[:,pd.np.r_[0:93, 121:169]]

,user_id,seller_id,label,age_range,gender,userbeforesale0,userbeforesale1,userbeforesale2,userbeforesale3,useronsale0,useronsale1,useronsale2,useronsale3,user_ratio_0_to_2_before_sale,user_ratio_3_to_2_before_sale,user_ratio_0_to_2_on_sale,user_ratio_3_to_2_on_sale,active_days,purchase_days,useractivity5,useractivity6,useractivity7,useractivity8,useractivity9,useractivity10,useractivity11,userpurchase5,userpurchase6,userpurchase7,userpurchase8,userpurchase9,userpurchase10,userpurchase11,useravgactivity5,useravgactivity6,useravgactivity7,useravgactivity8,useravgactivity9,useravgactivity10,useravgactivity11,useravgpurchase5,useravgpurchase6,useravgpurchase7,useravgpurchase8,useravgpurchase9,useravgpurchase10,useravgpurchase11,user_unique_brands_before_sale,user_unique_categories_before_sale,user_unique_items_before_sale,user_unique_sellers_before_sale,user_unique_brands_purchased_before_sale,user_unique_categories_purchased_before_sale,user_unique_items_purchased_before_sale,user_unique_sellers_purchased_before_sale,user_unique_brands_on_sale,user_unique_categories_on_sale,user_unique_items_on_sale,user_unique_sellers_on_sale,user_unique_brands_purchased_on_sale,user_unique_categories_purchased_on_sale,user_unique_items_purchased_on_sale,user_unique_sellers_purchased_on_sale,user_all_month_purchase,user_purchase_only_11,sellerbeforesale0,sellerbeforesale1,sellerbeforesale2,sellerbeforesale3,selleronsale0,selleronsale1,selleronsale2,selleronsale3,userseller0,userseller1,userseller2,userseller3,age_avg_activity,age_avg_purchase,unique_users_age_norm,gender_avg_activity,gender_avg_purchase,unique_users_gender_norm,age_gender,age_gender_avg_activity,age_gender_avg_purchase,unique_users_age_gender_norm,age_gender_active_days_norm,age_gender_purchase_days_norm,seller_ratio_0_to_2_before_sale,seller_ratio_3_to_2_before_sale,seller_ratio_0_to_2_on_sale,seller_ratio_3_to_2_on_sale,seller_unique_brands_before_sale,seller_unique_categories_before_sale,seller_unique_items_before_sale,seller_unique_users_before_sale,seller_unique_brands_purchased_before_sale,seller_unique_categories_purchased_before_sale,seller_unique_items_purchased_before_sale,seller_unique_users_purchased_before_sale,seller_unique_brands_on_sale,seller_unique_categories_on_sale,seller_unique_items_on_sale,seller_unique_users_on_sale,seller_unique_brands_purchased_on_sale,seller_unique_categories_purchased_on_sale,seller_unique_items_purchased_on_sale,seller_unique_users_purchased_on_sale,user_merchant_pair,usersellerbeforesale0,usersellerbeforesale1,usersellerbeforesale3,userselleronsale0,userselleronsale1,userselleronsale2,userselleronsale3,um_unique_brands_on_sale,um_unique_categories_on_sale,um_unique_items_on_sale,um_unique_brands_purchased_on_sale,um_unique_categories_purchased_on_sale,um_unique_items_purchased_on_sale,um_activity_before_sale,activity_diff_from_1111,activity_diff_from_second_last,purchase_diff_from_1111,purchase_diff_from_second_last,seller_count_of_label-1,brand_intersection_user_before_on_sale,brand_intersection_user_before_and_seller,ratio_brand_int_union_user_before_on_sale,ratio_brand_int_union_user_before_seller,cat_intersection_user_before_on_sale,cat_intersection_user_before_and_seller,ratio_cat_int_union_user_before_on_sale,ratio_cat_int_union_user_before_seller,item_intersection_user_before_on_sale,item_intersection_user_before_and_seller,ratio_item_int_union_user_before_on_sale,ratio_item_int_union_user_before_seller
130649,1,1019,1,3.0,1.0,14.0,0.0,2.0,0.0,13.0,0.0,4.0,0.0,7.000000,0.00,3.250000,0.0,4,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.484848,0.515152,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.666667,0.0,0.000000,0.000000,0.000000,0.000000,4.000000,17.000000,0.0,0.0,0.000000,0.0,0.0,1.0,4.0,5,3,8,5,2.0,2.0,2.0,2.0,4,3,4,4,1,1,1,1,0.0,0.000000,4836.0,4.0,707.0,447.0,1167.0,1.0,234.0,37.0,10.0,0.0,4.0,0.0,132.987954,7.753050,1.000000,99.741350,6.668382,0.425959,3.0_1.0,102.861818,6.632899,0.445214,0.316205,0.36307

In [32]:
#Sort
df_combined = df_combined.sort_values(['user_id','seller_id'])
df_train = df_train.sort_values(['user_id','seller_id'])

final_feat = (df_combined.iloc[:,pd.np.r_[0:93, 121:169]]\
              .drop(['user_id', 'seller_id', 'label','age_gender', 'user_merchant_pair'], axis=1))

final_label = df_train['label']

In [33]:
model_xgboost(final_feat,final_label)

Iteration Complete!!
Iteration Complete!!
Iteration Complete!!
Iteration Complete!!
Iteration Complete!!
Mean auc for validation =  0.6747761287680629 	 Mean auc for valid baseline = 0.6498
Mean auc for train =  0.7782986218836969 	 Mean auc for train baseline = 0.6629
                                            feature  importance_abs  importance_norm
135         ratio_item_int_union_user_before_seller        0.000000         0.000000
79                         unique_users_gender_norm        0.000000         0.000000
106                           usersellerbeforesale1        0.000000         0.000000
109                               userselleronsale1        0.000000         0.000000
110                               userselleronsale2        0.000000         0.000000
123                         seller_count_of_label-1        0.000000         0.000000
125       brand_intersection_user_before_and_seller        0.000000         0.000000
128            cat_intersection_user_before_on_sal

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import seaborn as sns
import matplotlib.pyplot as plt

def roc_auc(X_test,y_test,y_pred,model):
#    confusion_mat = confusion_matrix(y_test, y_pred)
#    print('Confusion_Matrix: ',confusion_mat,sep='\n')
#    print()
#    print('Accuracy of classifier on test set: {:.2f}'.format(model.score(X_test, y_test)))
#    print()
   
#    # Classification Report
#    print('Classification Report: ')
#    print(classification_report(y_test, y_pred))
#    y_score = model.predict_proba(X_test)[:,1]              # Predicted probability score
#    fpr, tpr, thresholds = roc_curve(y_test, y_score)
#    plt.figure()
   
   # AuC Score & plotting of AuC curve
   plt.plot(fpr, tpr, label='AuC score (area = %0.2f)' % roc_auc_score(y_test, y_score))
   plt.plot([0, 1], [0, 1],'r--')
   plt.xlim([0.0, 1.0])
   plt.ylim([0.0, 1.05])
   plt.xlabel('False Positive Rate')
   plt.ylabel('True Positive Rate')
   plt.title('Receiver operating characteristic')
   plt.legend(loc="lower right")

# FOR PREDICTION VALUES

In [34]:
STOP

NameError: name 'STOP' is not defined

In [35]:
def model_xgboost_pred(x, y):
    
    model = xgb.XGBClassifier()
    
    skf = StratifiedKFold(n_splits = 5, random_state = 2)
    
    AUC_valid = []
    AUC_train = []
    
    for train_index, valid_index in skf.split(x, y):
    
        #print("TRAIN:", train_index, "VALID:", valid_index)
    
        X_train1, X_valid1 = x.iloc[train_index], x.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
        X_train = X_train1.drop(['user_id', 'seller_id', 'label','age_gender', 'user_merchant_pair'], axis=1)
        X_valid = X_valid1.drop(['user_id', 'seller_id', 'label','age_gender', 'user_merchant_pair'], axis=1)
    
        model.fit(X_train, y_train, eval_metric='auc')
        
        auc_valid = roc_auc_score(y_valid, model.predict_proba(X_valid)[:,1])
        #print(auc_valid)
        AUC_valid.append(auc_valid)
        Å
        X_train1['prediction'] = model.predict_proba(X_train)[:,1]
        X_valid1['prediction'] = model.predict_proba(X_valid)[:,1]
        
        auc_train = roc_auc_score(y_train, model.predict_proba(X_train)[:,1])
        #print(auc_train)
        AUC_train.append(auc_train)
        
        break
        
    def mean(z):
        return sum(z)/len(z)
    
    print("Mean auc for validation = ", mean(AUC_valid), "\t Mean auc for valid baseline = 0.6498")
    print("Mean auc for train = ", mean(AUC_train), "\t Mean auc for train baseline = 0.6629")
        
    feat_imp_abs = model.feature_importances_
    feat_imp_abs = pd.Series(data=feat_imp_abs,dtype='float')
    feat_imp_norm = feat_imp_abs.div(feat_imp_abs.max())*100    
        
    feature_importance = list(final_feat)
    feature_importance = pd.DataFrame(feature_importance)
    feature_importance['importance_abs'] = feat_imp_abs
    feature_importance['importance_norm'] = feat_imp_norm
    feature_importance = feature_importance.rename(columns={0: "feature"})
    
    print(feature_importance.sort_values('importance_norm'))
    
    return X_train1, X_valid1

In [ ]:
#Sort
df_combined = df_combined.sort_values(['user_id','seller_id'])
df_train = df_train.sort_values(['user_id','seller_id'])

final_feat = (df_combined.drop(['user_id', 'seller_id', 'label','age_gender', 'user_merchant_pair'], axis=1))

final_label = df_train['label']

# To add prediction to features df

In [ ]:
# X_train1,X_valid1 = model_xgboost(df_combined, final_label)

# prediction = pd.concat([X_train1,X_valid1],ignore_index = True)

# cols = prediction.columns.tolist()

# cols = cols[-1:] + cols[:-1]

# prediction = prediction[cols]

# prediction = prediction.sort_values('prediction', ascending = False)

# prediction.to_csv("prediction1.csv",index=False)

# Grid Search for Parameter Tuning

In [ ]:
# gs = GridSearchCV(xgb_model, {"model__max_depth": [5, 10],
#                               "model__min_child_weight": [5, 10],
#                               "model__n_estimators": [25]},
#                   n_jobs=-1, cv=5, scoring="accuracy")

# gs.fit(X_train, y_train)

In [ ]:
# print(gs.best_params_)
# print(gs.best_score_)
# xgb_model.set_params(**gs.best_params_)
# xgb_model.fit(X_train, y_train)

In [ ]:
roc_auc_score(y_test, xgb_model.predict_proba(X_test)[:,1])

In [ ]:
eli5.show_weights(xgb_model, top = 200)

In [50]:
model_eli5 = xgb.XGBClassifier(max_depth = 6)

skf = StratifiedKFold(n_splits = 2, random_state = 2)

In [51]:
for train_index, valid_index in skf.split(final_feat, final_label):
    
    X_train, X_valid = final_feat.iloc[train_index], final_feat.iloc[valid_index]
    y_train, y_valid = final_label.iloc[train_index], final_label.iloc[valid_index]

In [52]:
model_eli5.fit(X_train, y_train, eval_metric='auc')

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [53]:
features = list(final_feat.columns)

In [54]:
eli5.show_prediction(xgb_model, 
                     X_test.iloc[[4]],
                     feature_names=features, show_feature_values=True, top = 50)

NameError: name 'xgb_model' is not defined

In [ ]:
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report
# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import roc_curve
# import seaborn as sns
# import matplotlib.pyplot as plt

# def roc_auc(x,y,y_pred,model):
#    confusion_mat = confusion_matrix(y, y_pred)
#    print('Confusion_Matrix: ',confusion_mat,sep='\n')
#    print()
#    print('Accuracy of classifier on test set: {:.2f}'.format(model.score(x, y)))
#    print()
   
#    # Classification Report
#    print('Classification Report: ')
#    print(classification_report(y, y_pred))
#    y_score = model.predict_proba(X_test)[:,1]              # Predicted probability score
#    fpr, tpr, thresholds = roc_curve(y, y_score)
#    plt.figure()
   
#    # AuC Score & plotting of AuC curve
#    plt.plot(fpr, tpr, label='AuC score (area = %0.2f)' % roc_auc_score(y, y_score))
#    plt.plot([0, 1], [0, 1],'r--')
#    plt.xlim([0.0, 1.0])
#    plt.ylim([0.0, 1.05])
#    plt.xlabel('False Positive Rate')
#    plt.ylabel('True Positive Rate')
#    plt.title('Receiver operating characteristic')
#    plt.legend(loc="lower right")



In [ ]:
roc_auc(xval1,yval1,y_pred,regressor)

In [ ]:
features = list(final_feat.columns)

In [49]:
eli5.show_prediction(xgb_model, 
                     X_test.iloc[[4]],
                     feature_names=features, show_feature_values=True, top = 20)

NameError: name 'xgb_model' is not defined

In [ ]:
eli5.show_prediction(xgb_model, 
                     X_test.iloc[[10]],
                     feature_names=features, show_feature_values=True, top = 20)

In [ ]:
eli5.show_prediction(xgb_model, 
                     X_test.iloc[[78]],
                     feature_names=features, show_feature_values=True, top = 20)

In [ ]:
eli5.show_prediction(xgb_model, 
                     X_test.iloc[[21]],
                     feature_names=features, show_feature_values=True, top = 50)

In [ ]:
eli5.show_prediction(xgb_model, 
                     X_test.iloc[[1570]],
                     feature_names=features, show_feature_values=True, top = 20)

In [ ]:
eli5.show_prediction(xgb_model, 
                     X_test.iloc[[5570]],
                     feature_names=features, show_feature_values=True, top = 20)

In [ ]:
eli5.show_prediction(xgb_model, 
                     X_test.iloc[[5570]],
                     feature_names=features, show_feature_values=True, top = 20)

In [ ]:
X_test

In [ ]:
from lime.lime_tabular import LimeTabularExplainer

In [ ]:
xgb_predict_proba = xgb_model.predict_proba(X_test)[:,1]

In [ ]:
explanation = explainer.explain_instance(final_feat, xgb_predict_proba, num_features=5)

In [ ]:
pip install shap

In [ ]:
import shap
# Need to load JS vis in the notebook
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(xgb_model)

In [ ]:
explainer

In [ ]:
shap_values = explainer.shap_values(X_train, y_train)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[0,:], X_train.iloc[0,:])

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[5,:], X_train.iloc[5,:])

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[1000,:], X_train.iloc[1000,:])

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[50,:], X_train.iloc[50,:])

# Logistic Regression

In [ ]:
# from sklearn.linear_model import LogisticRegression

In [ ]:
# model = LogisticRegression(random_state=0)

In [ ]:
# skf = StratifiedKFold(n_splits = 5, random_state = 0)

# for train_index, valid_index in skf.split(final_feat, final_label):
#     print("TRAIN:", train_index, "VALID:", valid_index)
#     X_train, X_valid = final_feat.iloc[train_index], final_feat.iloc[valid_index]
#     y_train, y_valid = final_label.iloc[train_index], final_label.iloc[valid_index]
    
#     model.fit(X_train, y_train)
    
#     print( roc_auc_score(y_valid, model.predict_proba(X_valid)[:,1]) )
    
#     #accuracy = model.score(X_valid, y_valid)
    
#     #y_pred = model.predict(X_valid)
    
#     #print(accuracy)

In [ ]:
# y_pred = model.predict(X_valid)

In [ ]:
# confusion_matrix(y_valid, y_pred)
